# CIFAR-100 Image Classification using Pretrained ResNet-34
This notebook applies transfer learning using a pretrained ResNet-34 model to classify images from the CIFAR-100 dataset. We improve performance using data augmentation, layer freezing, dropout, a learning rate scheduler, and qualitative analysis.

In [ ]:
# Install required packages
!pip install -q torchmetrics lightning-utilities

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.9/960.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.7 MB/s eta 0:00:00


## Importing Required Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torchmetrics
from torchmetrics import Accuracy, ConfusionMatrix
from torchvision.utils import make_grid

# Set computation device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cpu


## Data Preparation and Augmentation
Apply transformations to improve generalization, and prepare data loaders.

In [ ]:
# Define transformations for training and test data
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),       # Random crop to increase spatial robustness
    transforms.RandomHorizontalFlip(),          # Horizontal flip for augmentation
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Color jitter for additional variation
    transforms.ToTensor()                       # Convert PIL image to tensor
])

transform_test = transforms.Compose([
    transforms.ToTensor()  # No augmentation for test/validation
])

# Load CIFAR-100 dataset
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

# Split training into train and validation sets
train_set, val_set = torch.utils.data.random_split(train_dataset, [40000, 10000])

# Create DataLoaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 169M/169M [00:02<00:00, 63.8MB/s]


## Model Setup: ResNet-34 with Transfer Learning
We load a pretrained ResNet-34, freeze early layers, and replace the classifier head.

In [ ]:
# Load pretrained ResNet-34 model
model = models.resnet34(weights='IMAGENET1K_V1')
# model = models.resnet50(weights='IMAGENET1K_V1')

# Freeze early layers to retain pretrained features
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last few layers for fine-tuning
for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

# Replace the final fully connected layer for CIFAR-100
model.fc = nn.Sequential(
    nn.Dropout(0.4),  # Dropout for regularization
    nn.Linear(model.fc.in_features, 100)  # Output layer for 100 classes
)

# Move model to appropriate device
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 83.7MB/s]


## Define Loss, Optimizer and Learning Rate Scheduler

In [ ]:
# Cross entropy loss for multi-class classification
criterion = nn.CrossEntropyLoss()

# AdamW optimizer with weight decay for regularization
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
# optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # Adam optimizer with learning rate 0.001

# Learning rate scheduler to reduce LR over time
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

## Training and Validation Functions

In [ ]:
# Train the model for one epoch
def train_one_epoch():
    model.train()
    acc = Accuracy(task='multiclass', num_classes=100).to(device)
    total_loss = 0
    for X, Y in train_loader:
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, Y)
        loss.backward()
        optimizer.step()
        acc.update(outputs.argmax(1), Y)
        total_loss += loss.item() * X.size(0)
    return total_loss / len(train_loader.dataset), acc.compute().item()

# Evaluate the model
def validate_one_epoch(loader):
    model.eval()
    acc = Accuracy(task='multiclass', num_classes=100).to(device)
    total_loss = 0
    with torch.no_grad():
        for X, Y in loader:
            X, Y = X.to(device), Y.to(device)
            outputs = model(X)
            loss = criterion(outputs, Y)
            acc.update(outputs.argmax(1), Y)
            total_loss += loss.item() * X.size(0)
    return total_loss / len(loader.dataset), acc.compute().item()

## Train the Model

In [ ]:
history = pd.DataFrame()
epochs = 60

for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch()
    val_loss, val_acc = validate_one_epoch(val_loader)
    scheduler.step()  # Adjust learning rate

    print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")
    history = pd.concat([history, pd.DataFrame({
        'epoch': [epoch],
        'train_loss': [train_loss],
        'train_acc': [train_acc],
        'val_loss': [val_loss],
        'val_acc': [val_acc]
    })], ignore_index=True)

## Visualize Training Progress

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history['epoch'], history['train_loss'], label='Train Loss')
plt.plot(history['epoch'], history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(history['epoch'], history['train_acc'], label='Train Acc')
plt.plot(history['epoch'], history['val_acc'], label='Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Final Evaluation on Test Set

In [ ]:
model.eval()
test_acc = Accuracy(task='multiclass', num_classes=100)
conf_matrix = ConfusionMatrix(task='multiclass', num_classes=100)

with torch.no_grad():
    for X, Y in test_loader:
        preds = model(X.to(device)).argmax(1)
        test_acc.update(preds.cpu(), Y)
        conf_matrix.update(preds.cpu(), Y)

print("Test Accuracy:", test_acc.compute().item())